# Soil Moisture Data Sources

This notebook implements standardized retrievers for multiple soil moisture data sources. Each source has unique characteristics:

## Data Source Characteristics

### Data Assimilation 

### ERA5
- **Provider**: ECMWF (European Centre for Medium-Range Weather Forecasts)
- **Resolution**: 0.1° x 0.1° (approximately 9km)
- **Temporal Coverage**: 1979-present
- **Update Frequency**: Monthly updates, with 2-3 month delay
- **Key Features**: High spatial resolution, consistent reanalysis

### GLDAS
- **Provider**: NASA GSFC
- **Resolution**: 0.25° x 0.25°
- **Temporal Coverage**: 2000-present
- **Update Frequency**: 3-hourly
- **Key Features**: Global coverage, multiple soil layers

### NLDAS
- **Provider**: NASA/NOAA
- **Resolution**: 0.125° x 0.125°
- **Temporal Coverage**: 1979-present
- **Update Frequency**: Hourly
- **Key Features**: North American focus, high temporal resolution

### FLDAS
- **Provider**: NASA GSFC
- **Resolution**: 0.1° x 0.1°
- **Temporal Coverage**: 1982-present
- **Update Frequency**: Monthly
- **Key Features**: Africa-focused land data assimilation

### MERRA-2
- **Provider**: NASA GMAO
- **Resolution**: 0.5° x 0.625°
- **Temporal Coverage**: 1980-present
- **Update Frequency**: Monthly
- **Key Features**: Comprehensive atmospheric reanalysis

### Remote Sensing 

### SMAP
- **Provider**: NASA
- **Resolution**: 9km x 9km
- **Temporal Coverage**: 2015-present
- **Update Frequency**: 3-hourly
- **Key Features**: Direct satellite observations, high accuracy


In [ ]:
#import necessary packages 
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import cdsapi
import netCDF4
import earthaccess
import os
import tempfile
import sys
import json
import urllib3
import certifi
import requests
from time import sleep
from http.cookiejar import CookieJar
import urllib.request
from urllib.parse import urlencode
import getpass
from datetime import datetime
import h5py
from tqdm.auto import tqdm
import concurrent.futures
import warnings
from typing import Tuple, Optional,List, Dict
from pathlib import Path
import ftplib
import ssl
import json
import sys
import time



In [3]:
#calculate statistics for the datasets 
def calculate_statistics(data, var_name, var_attrs):
    """
    Calculate statistics for a variable based on its data type
    
    Parameters:
    -----------
    data : np.ndarray
        The data array to analyze
    var_name : str
        Name of the variable
    var_attrs : dict
        Variable attributes
        
    Returns:
    --------
    dict
        Dictionary containing the statistics
    """
    # Check if data is datetime type
    if np.issubdtype(data.dtype, np.datetime64):
        return {
            'type': 'datetime',
            'min': data.min(),
            'max': data.max(),
            'shape': data.shape
        }
    
    # For numeric data
    valid_data = data[~np.isnan(data)]
    if len(valid_data) > 0:
        return {
            'type': 'numeric',
            'mean': np.nanmean(data),
            'median': np.nanmedian(data),
            'std': np.nanstd(data),
            'var': np.nanvar(data),
            'min': np.nanmin(data),
            'max': np.nanmax(data),
            'valid_points': len(valid_data),
            'missing_points': np.sum(np.isnan(data)),
            'coverage': (len(valid_data) / data.size * 100),
            'shape': data.shape
        }
    
    return {
        'type': 'empty',
        'shape': data.shape
    }

In [4]:
# ERA5 data retrieval with progress tracking
def get_era5_data(dataset, request, output_file):
    """
    Retrieve ERA5 data with progress tracking using a custom download approach
    
    Parameters:
    -----------
    dataset : str
        Name of the ERA5 dataset
    request : dict
        Request parameters for ERA5 data
    output_file : str
        Path to save the downloaded file
    
    Returns:
    --------
    xarray.Dataset
        The loaded dataset with ERA5 data
    """
    print("\nRetrieving ERA5 data...")
    print(f"Dataset: {dataset}")
    print(f"Time range: {request['year']}-{request['month']}")
    print(f"Spatial bounds: {request['area']}")
    
    try:
        client = cdsapi.Client()
        
        # First, submit the request and get the result
        print("Submitting request to ERA5...")
        result = client.retrieve(dataset, request)
        
        # Download the file with manual progress tracking
        print("\nDownloading data...")
        with open(output_file, 'wb') as f:
            result.download(output_file)
        
        # Get file size after download
        file_size = os.path.getsize(output_file)
        print(f"Download complete. File size: {file_size/1024/1024:.2f} MB")
        
        # Load and analyze the downloaded data
        print("\nLoading dataset...")
        ds = xr.open_dataset(output_file)
        
        # Print dataset information
        print("\nDataset Information:")
        print("-" * 50)
        print(f"Dimensions: {dict(ds.dims)}")
        print("\nVariables:")
        for var in ds.data_vars:
            print(f"\nVariable: {var}")
            data = ds[var].values
            valid_data = data[~np.isnan(data)]
            
            # Get variable attributes
            attrs = ds[var].attrs
            units = attrs.get('units', 'unknown')
            long_name = attrs.get('long_name', var)
            
            print(f"Description: {long_name}")
            print(f"Units: {units}")
            print(f"Shape: {data.shape}")
            
            # Calculate statistics
            if len(valid_data) > 0:
                print("Statistics:")
                print(f"  Mean:     {np.nanmean(data):.4f}")
                print(f"  Median:   {np.nanmedian(data):.4f}")
                print(f"  Std Dev:  {np.nanstd(data):.4f}")
                print(f"  Variance: {np.nanvar(data):.4f}")
                print(f"  Min:      {np.nanmin(data):.4f}")
                print(f"  Max:      {np.nanmax(data):.4f}")
                print(f"  Valid Points:    {len(valid_data):,}")
                print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                print(f"  Data Coverage:   {(len(valid_data) / data.size * 100):.1f}%")
            else:
                print("No valid data points found")
        
        return ds
        
    except Exception as e:
        print(f"Error retrieving ERA5 data: {str(e)}")
        raise

In [5]:
#NLDAS data request 
def get_nldas_data(start_date, end_date, lat_bounds, lon_bounds):
    """
    Retrieve NLDAS soil moisture data with progress tracking
    
    Parameters:
    -----------
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    lat_bounds : tuple
        (min_lat, max_lat) for the region of interest
    lon_bounds : tuple
        (min_lon, max_lon) for the region of interest
    
    Returns:
    --------
    xarray.Dataset
        Combined dataset with NLDAS soil moisture data
    """
    print("\nRetrieving NLDAS soil moisture data...")
    print(f"Time range: {start_date} to {end_date}")
    print(f"Spatial bounds: lat {lat_bounds}, lon {lon_bounds}")
    
    try:
        # Authenticate with NASA Earthdata
        auth = earthaccess.login()
        
        # Search for granules
        print("\nSearching for NLDAS granules...")
        granules = earthaccess.search_data(
            short_name="NLDAS_NOAH0125_H",
            version="2.0",
            temporal=(start_date, end_date),
            bounding_box=(lon_bounds[0], lat_bounds[0], lon_bounds[1], lat_bounds[1])
        )
        
        if not granules:
            raise ValueError("No NLDAS granules found for the specified parameters")
            
        print(f"Found {len(granules)} granules")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            # Download files
            print("\nDownloading granules...")
            downloaded_files = earthaccess.download(
                granules,
                local_path=temp_dir
            )
            
            if not downloaded_files:
                raise ValueError("Failed to download any granules")
            
            print(f"Successfully downloaded {len(downloaded_files)} files")
            
            # Process files
            print("\nProcessing downloaded files...")
            datasets = []
            
            # Use tqdm for processing progress
            for file_path in tqdm(downloaded_files, desc="Processing files", unit="file"):
                try:
                    ds = xr.open_dataset(file_path)
                    
                    # Select only soil moisture variables (SoilM_*)
                    soil_vars = [var for var in ds.data_vars if 'SoilM_' in var]
                    if not soil_vars:
                        print(f"Warning: No soil moisture variables found in {os.path.basename(file_path)}")
                        continue
                    ds = ds[soil_vars]
                    
                    # Apply spatial subsetting
                    if 'lat' in ds.dims:
                        ds = ds.sel(lat=slice(lat_bounds[0], lat_bounds[1]))
                    if 'lon' in ds.dims:
                        ds = ds.sel(lon=slice(lon_bounds[0], lon_bounds[1]))
                    
                    datasets.append(ds)
                except Exception as e:
                    print(f"Warning: Failed to process file {os.path.basename(file_path)}: {str(e)}")
                    continue
            
            if not datasets:
                raise ValueError("No valid soil moisture data found in downloaded files")
            
            # Combine datasets
            print("\nCombining datasets...")
            combined_ds = xr.concat(datasets, dim='time')
            
            # Print dataset information
            print("\nDataset Information:")
            print("-" * 50)
            print(f"Time range: {combined_ds.time.values[0]} to {combined_ds.time.values[-1]}")
            print(f"Number of timesteps: {len(combined_ds.time)}")
            print(f"Dimensions: {dict(combined_ds.sizes)}")
            
            # Print statistics for soil moisture variables
            print("\nSoil Moisture Statistics:")
            print("-" * 50)
            
            for var in combined_ds.data_vars:
                print(f"\nVariable: {var}")
                data = combined_ds[var].values
                
                # Get variable attributes
                attrs = combined_ds[var].attrs
                units = attrs.get('units', 'kg/m^2')
                long_name = attrs.get('long_name', var)
                
                print(f"Description: {long_name}")
                print(f"Units: {units}")
                print(f"Shape: {data.shape}")
                
                # Calculate statistics for numeric data
                if np.issubdtype(data.dtype, np.number):
                    valid_data = data[~np.isnan(data)]
                    if len(valid_data) > 0:
                        print("Statistics:")
                        print(f"  Mean:     {np.nanmean(data):.4f}")
                        print(f"  Median:   {np.nanmedian(data):.4f}")
                        print(f"  Std Dev:  {np.nanstd(data):.4f}")
                        print(f"  Min:      {np.nanmin(data):.4f}")
                        print(f"  Max:      {np.nanmax(data):.4f}")
                        print(f"  Valid Points:    {len(valid_data):,}")
                        print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                        print(f"  Data Coverage:   {(len(valid_data) / data.size * 100):.1f}%")
                    else:
                        print("No valid numeric data found")
            
            return combined_ds
            
    except Exception as e:
        print(f"\nError retrieving NLDAS soil moisture data: {str(e)}")
        raise

In [6]:
# GLDAS data retrieval with progress tracking
def get_gldas_data(start_date, end_date, lat_bounds, lon_bounds):
    """
    Retrieve GLDAS soil moisture data with progress tracking
    
    Parameters:
    -----------
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    lat_bounds : tuple
        (min_lat, max_lat) for the region of interest
    lon_bounds : tuple
        (min_lon, max_lon) for the region of interest
        
    Returns:
    --------
    xarray.Dataset
        Combined dataset with GLDAS soil moisture data
    """
    print("\nRetrieving GLDAS soil moisture data...")
    print(f"Time range: {start_date} to {end_date}")
    print(f"Spatial bounds: lat {lat_bounds}, lon {lon_bounds}")
    
    try:
        auth = earthaccess.login()
        
        # Search for granules
        granules = earthaccess.search_data(
            short_name="GLDAS_NOAH025_3H",
            version="2.1",
            temporal=(start_date, end_date),
            bounding_box=(lon_bounds[0], lat_bounds[0], lon_bounds[1], lat_bounds[1])
        )
        
        if not granules:
            raise ValueError("No GLDAS granules found for the specified parameters")
            
        print(f"Found {len(granules)} granules")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            # Download files
            print("\nDownloading granules...")
            downloaded_files = earthaccess.download(
                granules,
                local_path=temp_dir
            )
            
            if not downloaded_files:
                raise ValueError("Failed to download any granules")
                
            print(f"Successfully downloaded {len(downloaded_files)} files")
            
            # Process files
            print("\nProcessing downloaded files...")
            datasets = []
            
            for file_path in tqdm(downloaded_files, desc="Processing files", unit="file"):
                try:
                    ds = xr.open_dataset(file_path)
                    
                    # Print dimensions and variables for first file
                    if len(datasets) == 0:
                        print(f"\nFile: {os.path.basename(file_path)}")
                        print("Dimensions:", list(ds.dims))
                        print("Data variables:", list(ds.data_vars))
                    
                    # Select soil moisture variables (SoilMoi*)
                    soil_vars = [var for var in ds.data_vars if 'SoilMoi' in var]
                    if not soil_vars:
                        print(f"Warning: No soil moisture variables found in {os.path.basename(file_path)}")
                        continue
                    ds = ds[soil_vars]
                    
                    # Apply spatial subsetting
                    if 'lat' in ds.dims:
                        ds = ds.sel(lat=slice(lat_bounds[0], lat_bounds[1]))
                    if 'lon' in ds.dims:
                        ds = ds.sel(lon=slice(lon_bounds[0], lon_bounds[1]))
                    
                    datasets.append(ds)
                except Exception as e:
                    print(f"Warning: Failed to process file {os.path.basename(file_path)}: {str(e)}")
                    continue
            
            if not datasets:
                raise ValueError("No valid soil moisture data found in downloaded files")
            
            # Combine datasets
            print("\nCombining datasets...")
            combined_ds = xr.concat(datasets, dim='time')
            
            # Print dataset information
            print("\nDataset Information:")
            print("-" * 50)
            print(f"Time range: {combined_ds.time.values[0]} to {combined_ds.time.values[-1]}")
            print(f"Number of timesteps: {len(combined_ds.time)}")
            print(f"Dimensions: {dict(combined_ds.sizes)}")
            
            # Print statistics for soil moisture variables
            print("\nSoil Moisture Statistics:")
            print("-" * 50)
            
            for var in combined_ds.data_vars:
                print(f"\nVariable: {var}")
                data = combined_ds[var].values
                
                # Get variable attributes
                attrs = combined_ds[var].attrs
                units = attrs.get('units', 'kg/m^2')
                long_name = attrs.get('long_name', var)
                
                print(f"Description: {long_name}")
                print(f"Units: {units}")
                print(f"Shape: {data.shape}")
                
                # Calculate statistics
                if np.issubdtype(data.dtype, np.number):
                    valid_data = data[~np.isnan(data)]
                    if len(valid_data) > 0:
                        print("Statistics:")
                        print(f"  Mean:     {np.nanmean(data):.4f}")
                        print(f"  Median:   {np.nanmedian(data):.4f}")
                        print(f"  Std Dev:  {np.nanstd(data):.4f}")
                        print(f"  Min:      {np.nanmin(data):.4f}")
                        print(f"  Max:      {np.nanmax(data):.4f}")
                        print(f"  Valid Points:    {len(valid_data):,}")
                        print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                        print(f"  Data Coverage:   {(len(valid_data) / data.size * 100):.1f}%")
                    else:
                        print("No valid numeric data found")
            
            return combined_ds
            
    except Exception as e:
        print(f"\nError retrieving GLDAS soil moisture data: {str(e)}")
        raise

In [7]:
# FLDAS data retrieval with progress tracking
def get_fldas_data(start_date, end_date, lat_bounds, lon_bounds):
    """
    Retrieve FLDAS soil moisture data with progress tracking
    
    Parameters:
    -----------
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    lat_bounds : tuple
        (min_lat, max_lat) for the region of interest
    lon_bounds : tuple
        (min_lon, max_lon) for the region of interest
        
    Returns:
    --------
    xarray.Dataset
        Combined dataset with FLDAS soil moisture data
    """
    print("\nRetrieving FLDAS soil moisture data...")
    print(f"Time range: {start_date} to {end_date}")
    print(f"Spatial bounds: lat {lat_bounds}, lon {lon_bounds}")
    
    try:
        auth = earthaccess.login()
        
        granules = earthaccess.search_data(
            short_name="FLDAS_NOAH01_C_GL_M",
            version="001",
            temporal=(start_date, end_date),
            bounding_box=(lon_bounds[0], lat_bounds[0], lon_bounds[1], lat_bounds[1])
        )
        
        if not granules:
            raise ValueError("No FLDAS granules found for the specified parameters")
            
        print(f"Found {len(granules)} granules")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            downloaded_files = earthaccess.download(
                granules,
                local_path=temp_dir
            )
            
            if not downloaded_files:
                raise ValueError("Failed to download any granules")
                
            print(f"Successfully downloaded {len(downloaded_files)} files")
            
            datasets = []
            for file_path in tqdm(downloaded_files, desc="Processing files", unit="file"):
                try:
                    ds = xr.open_dataset(file_path)
                    
                    # Print dimensions and variables for first file
                    if len(datasets) == 0:
                        print(f"\nFile: {os.path.basename(file_path)}")
                        print("Dimensions:", list(ds.dims))
                        print("Data variables:", list(ds.data_vars))
                    
                    # Select soil moisture variables (SoilMoi*cm_tavg)
                    soil_vars = [var for var in ds.data_vars if 'SoilMoi' in var and 'cm_tavg' in var]
                    if not soil_vars:
                        print(f"Warning: No soil moisture variables found in {os.path.basename(file_path)}")
                        continue
                    ds = ds[soil_vars]
                    
                    # Apply spatial subsetting using X/Y coordinates
                    if 'X' in ds.dims and 'Y' in ds.dims:
                        # Convert lat/lon bounds to X/Y if needed
                        ds = ds.sel(X=slice(lon_bounds[0], lon_bounds[1]),
                                  Y=slice(lat_bounds[0], lat_bounds[1]))
                    
                    datasets.append(ds)
                except Exception as e:
                    print(f"Warning: Failed to process file {os.path.basename(file_path)}: {str(e)}")
                    continue
            
            if not datasets:
                raise ValueError("No valid soil moisture data found in downloaded files")
            
            print("\nCombining datasets...")
            combined_ds = xr.concat(datasets, dim='time')
            
            print("\nDataset Information:")
            print("-" * 50)
            print(f"Time range: {combined_ds.time.values[0]} to {combined_ds.time.values[-1]}")
            print(f"Number of timesteps: {len(combined_ds.time)}")
            print(f"Dimensions: {dict(combined_ds.sizes)}")
            
            print("\nSoil Moisture Statistics:")
            print("-" * 50)
            
            for var in combined_ds.data_vars:
                print(f"\nVariable: {var}")
                data = combined_ds[var].values
                
                attrs = combined_ds[var].attrs
                units = attrs.get('units', 'kg/m^2')
                long_name = attrs.get('long_name', var)
                
                print(f"Description: {long_name}")
                print(f"Units: {units}")
                print(f"Shape: {data.shape}")
                
                if np.issubdtype(data.dtype, np.number):
                    valid_data = data[~np.isnan(data)]
                    if len(valid_data) > 0:
                        print("Statistics:")
                        print(f"  Mean:     {np.nanmean(data):.4f}")
                        print(f"  Median:   {np.nanmedian(data):.4f}")
                        print(f"  Std Dev:  {np.nanstd(data):.4f}")
                        print(f"  Min:      {np.nanmin(data):.4f}")
                        print(f"  Max:      {np.nanmax(data):.4f}")
                        print(f"  Valid Points:    {len(valid_data):,}")
                        print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                        print(f"  Data Coverage:   {(len(valid_data) / data.size * 100):.1f}%")
                    else:
                        print("No valid numeric data found")
            
            return combined_ds
            
    except Exception as e:
        print(f"\nError retrieving FLDAS soil moisture data: {str(e)}")
        raise

In [8]:
# SMAP data retrieval with progress tracking
def get_smap_data(start_date, end_date, lat_bounds, lon_bounds, max_files=10):
    """
    Retrieve SMAP L4 soil moisture data with missing value handling
    """
    print("\nRetrieving SMAP soil moisture data...")
    print(f"Time range: {start_date} to {end_date}")
    print(f"Spatial bounds: lat {lat_bounds}, lon {lon_bounds}")
    
    try:
        auth = earthaccess.login()
        
        granules = earthaccess.search_data(
            count=max_files,
            short_name="SPL4SMGP",
            temporal=(start_date, end_date),
            bounding_box=(lon_bounds[0], lat_bounds[0], lon_bounds[1], lat_bounds[1])
        )
        
        if not granules:
            raise ValueError("No SMAP granules found")
            
        print(f"Found {len(granules)} granules")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            downloaded_files = earthaccess.download(
                granules,
                local_path=temp_dir
            )
            
            if not downloaded_files:
                raise ValueError("Failed to download granules")
                
            datasets = []
            for file_path in tqdm(downloaded_files, desc="Processing"):
                try:
                    with h5py.File(file_path, 'r') as f:
                        if len(datasets) == 0:
                            print("\nFile structure:")
                            def print_structure(name, obj):
                                if isinstance(obj, h5py.Dataset):
                                    print(f"{name}:")
                                    print(f"  Shape: {obj.shape}")
                                    print(f"  Dtype: {obj.dtype}")
                                    if '_FillValue' in obj.attrs:
                                        print(f"  Fill Value: {obj.attrs['_FillValue']}")
                            f.visititems(print_structure)
                        
                        if 'Geophysical_Data' in f:
                            geo_data = f['Geophysical_Data']
                            sm_vars = ['sm_surface', 'sm_rootzone', 'sm_profile']
                            ds_dict = {}
                            
                            time_value = None
                            for attr in f.attrs.keys():
                                if 'time' in attr.lower():
                                    try:
                                        time_str = f.attrs[attr]
                                        if isinstance(time_str, bytes):
                                            time_str = time_str.decode('utf-8')
                                        time_value = pd.to_datetime(time_str)
                                        break
                                    except:
                                        continue
                            
                            if time_value is None:
                                time_value = pd.Timestamp(start_date)
                            
                            for var in sm_vars:
                                if var in geo_data:
                                    # Get the data and attributes
                                    data = geo_data[var][:]
                                    attrs = dict(geo_data[var].attrs)
                                    
                                    # Handle missing values
                                    # Check for _FillValue in attributes
                                    fill_value = attrs.get('_FillValue', -9999)
                                    # Replace both -9999 and the fill_value with NaN
                                    data = np.where(data == -9999, np.nan, data)
                                    if fill_value != -9999:
                                        data = np.where(data == fill_value, np.nan, data)
                                    
                                    y_size, x_size = data.shape
                                    coords = {
                                        'y': np.linspace(lat_bounds[0], lat_bounds[1], y_size),
                                        'x': np.linspace(lon_bounds[0], lon_bounds[1], x_size),
                                        'time': [time_value]
                                    }
                                    
                                    # Print statistics for this variable
                                    print(f"\nStatistics for {var}:")
                                    valid_data = data[~np.isnan(data)]
                                    if len(valid_data) > 0:
                                        print(f"  Mean:     {np.mean(valid_data):.4f}")
                                        print(f"  Std Dev:  {np.std(valid_data):.4f}")
                                        print(f"  Min:      {np.min(valid_data):.4f}")
                                        print(f"  Max:      {np.max(valid_data):.4f}")
                                        print(f"  Valid Points:    {len(valid_data):,}")
                                        print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                                        print(f"  Coverage:        {(len(valid_data) / data.size * 100):.1f}%")
                                        print(f"  Units:           {attrs.get('units', 'unknown')}")
                                    
                                    da = xr.DataArray(
                                        data[np.newaxis, :, :],
                                        dims=['time', 'y', 'x'],
                                        coords=coords,
                                        name=var,
                                        attrs=attrs
                                    )
                                    ds_dict[var] = da
                            
                            if ds_dict:
                                ds = xr.Dataset(ds_dict)
                                datasets.append(ds)
                                
                except Exception as e:
                    print(f"Warning: Failed to process {os.path.basename(file_path)}: {str(e)}")
            
            if not datasets:
                raise ValueError("No valid soil moisture data found")
            
            combined_ds = xr.concat(datasets, dim='time')
            print("\nRetrieved data summary:")
            print(f"Time period: {combined_ds.time.values[0]} to {combined_ds.time.values[-1]}")
            print("Variables:", list(combined_ds.data_vars))
            
            # Print final statistics for combined dataset
            print("\nFinal Dataset Statistics:")
            for var in combined_ds.data_vars:
                data = combined_ds[var].values
                valid_data = data[~np.isnan(data)]
                print(f"\n{var}:")
                if len(valid_data) > 0:
                    print(f"  Mean:     {np.mean(valid_data):.4f}")
                    print(f"  Std Dev:  {np.std(valid_data):.4f}")
                    print(f"  Min:      {np.min(valid_data):.4f}")
                    print(f"  Max:      {np.max(valid_data):.4f}")
                    print(f"  Valid Points:    {len(valid_data):,}")
                    print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                    print(f"  Coverage:        {(len(valid_data) / data.size * 100):.1f}%")
            
            return combined_ds
            
    except Exception as e:
        print(f"\nError: {str(e)}")
        raise

In [28]:
# MERRA-2 data retrieval with progress tracking
# Cell 1: Function Definition

def get_merra2_data(
    bbox: Tuple[float, float, float, float],
    date_range: Tuple[str, str],
    var_names: Optional[List[str]] = None,
    output_file: str = 'merra_2_soil_moisture_data.nc',
    product: str = 'M2T1NXLND_5.12.4'
) -> Optional[xr.Dataset]:
    """
    Fetch MERRA-2 data and perform basic statistical analysis.
    
    Parameters:
    -----------
    bbox : tuple
        Bounding box coordinates as (min_lon, min_lat, max_lon, max_lat)
        Valid ranges: longitude [-180, 180], latitude [-90, 90]
    
    date_range : tuple
        Start and end dates as ('YYYY-MM-DD', 'YYYY-MM-DD')
    
    var_names : list, optional
        List of variable names to fetch. If None, defaults to 
        ['SFMC', 'GWETTOP', 'PRMC', 'RZMC']
    
    output_file : str, optional
        Output filename for NetCDF data
        Default: 'merra_2_soil_moisture_data.nc'
    
    product : str, optional
        MERRA-2 product identifier
        Default: 'M2T1NXLND_5.12.4'
    
    Returns:
    --------
    xarray.Dataset or None
        Dataset containing the requested variables with statistics printed
        Returns None if the request fails
    
    Examples:
    --------
    >>> # Fetch data for New York State
    >>> ds = fetch_merra2_data(
    ...     bbox=(-79.77, 40.5, -71.85, 45.02),
    ...     date_range=('2020-01-01', '2020-01-31'),
    ...     var_names=['SFMC', 'PRMC']
    ... )
    """
    # Parameter validation
    try:
        # Validate and process bbox
        minlon, minlat, maxlon, maxlat = bbox
        if not (-180 <= minlon <= 180 and -180 <= maxlon <= 180):
            raise ValueError("Longitude must be between -180 and 180 degrees")
        if not (-90 <= minlat <= 90 and -90 <= maxlat <= 90):
            raise ValueError("Latitude must be between -90 and 90 degrees")
        if minlon >= maxlon or minlat >= maxlat:
            raise ValueError("Min values must be less than max values")

        # Validate and process dates
        start_date, end_date = date_range
        try:
            datetime.strptime(start_date, '%Y-%m-%d')
            datetime.strptime(end_date, '%Y-%m-%d')
        except ValueError:
            raise ValueError("Dates must be in YYYY-MM-DD format")
        
        if start_date > end_date:
            raise ValueError("Start date must be before end date")

        # Set default variables if none provided
        if var_names is None:
            var_names = ['SFMC', 'GWETTOP', 'PRMC', 'RZMC']
            print(f"Using default variables: {var_names}")

    except ValueError as e:
        print(f"Parameter validation failed: {e}")
        return None

    # Initialize urllib PoolManager and set base URL
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())
    url = 'https://disc.gsfc.nasa.gov/service/subset/jsonwsp'
    
    def get_http_data(request):
        hdrs = {'Content-Type': 'application/json',
                'Accept': 'application/json'}
        data = json.dumps(request)       
        r = http.request('POST', url, body=data, headers=hdrs)
        response = json.loads(r.data)   
        if response['type'] == 'jsonwsp/fault':
            print('API Error: faulty %s request' % response['methodname'])
            sys.exit(1)
        return response
    
    # Construct the subset request
    subset_request = {
        'methodname': 'subset',
        'type': 'jsonwsp/request',
        'version': '1.0',
        'args': {
            'role': 'subset',
            'start': start_date,
            'end': end_date,
            'box': [minlon, minlat, maxlon, maxlat],
            'crop': True,
            'data': [{'datasetId': product,
                      'variable': varName
                     } for varName in var_names]
        }
    }
    
    # Submit request and get job ID
    response = get_http_data(subset_request)
    myJobId = response['result']['jobId']
    print('Job ID:', myJobId)
    print('Initial status:', response['result']['Status'])
    
    # Monitor job status
    status_request = {
        'methodname': 'GetStatus',
        'version': '1.0',
        'type': 'jsonwsp/request',
        'args': {'jobId': myJobId}
    }
    
    while response['result']['Status'] in ['Accepted', 'Running']:
        time.sleep(5)
        response = get_http_data(status_request)
        status = response['result']['Status']
        percent = response['result']['PercentCompleted']
        print(f'Job status: {status} ({percent}% complete)')
    
    def download_netcdf(url, output_file):
        print(f"\nDownloading data to {output_file}...")
        try:
            response = requests.get(url)
            response.raise_for_status()
            with open(output_file, 'wb') as f:
                f.write(response.content)
            print(f"Successfully saved data to {output_file}")
            return True
        except requests.exceptions.RequestException as e:
            print(f"Error downloading data: {e}")
            return False
    
    # Get results and download data
    if response['result']['Status'] == 'Succeeded':
        print('Job Finished:', response['result']['message'])
        
        result = requests.get('https://disc.gsfc.nasa.gov/api/jobs/results/'+myJobId)
        try:
            result.raise_for_status()
            urls = result.text.split('\n')
            
            success = False
            for url in urls:
                if url.strip():
                    print("\nAttempting download from:", url)
                    success = download_netcdf(url, output_file)
                    if success:
                        break
            
            if success:
                # Load and analyze the data
                ds = xr.open_dataset(output_file)
                
                print("\n=== Dataset Information ===")
                print(f"Dimensions: {dict(ds.dims)}")
                print(f"\nVariables: {list(ds.data_vars)}")
                print(f"\nTime range: {ds.time.values[0]} to {ds.time.values[-1]}")
                print(f"Spatial extent: {ds.lon.values.min():.2f}°E to {ds.lon.values.max():.2f}°E, "
                      f"{ds.lat.values.min():.2f}°N to {ds.lat.values.max():.2f}°N")
                
                for var in ds.data_vars:
                    print(f"\n=== Statistics for {var} ===")
                    data = ds[var]
                    print(f"Shape: {data.shape}")
                    print(f"Missing values: {data.isnull().sum().values}")
                    print(f"Mean: {float(data.mean()):.4f}")
                    print(f"Min: {float(data.min()):.4f}")
                    print(f"Max: {float(data.max()):.4f}")
                    print(f"Standard deviation: {float(data.std()):.4f}")
                
                return ds
            else:
                print("Failed to download data. Please check your credentials and try again.")
                return None
            
        except requests.exceptions.RequestException as e:
            print('Error getting download URLs:', e)
            return None
    else:
        print('Job Failed:', response['fault']['code'])
        return None

In [31]:
#smos data request

def get_file_size(ftps: ftplib.FTP_TLS, filename: str) -> int:
    """Get file size in bytes"""
    try:
        return ftps.size(filename)
    except:
        return 0

def download_with_progress(ftps: ftplib.FTP_TLS, filename: str, local_path: str) -> bool:
    """Download file with progress bar"""
    file_size = get_file_size(ftps, filename)
    
    # Initialize progress bar
    pbar = tqdm(total=file_size, unit='B', unit_scale=True, 
                desc=f"Downloading {os.path.basename(filename)}")
    
    # Create callback for updating progress
    def callback(data):
        pbar.update(len(data))
        with open(local_path, 'ab') as f:
            f.write(data)
    
    try:
        # Clear existing file if any
        if os.path.exists(local_path):
            os.remove(local_path)
            
        # Download file
        ftps.retrbinary(f'RETR {filename}', callback)
        pbar.close()
        return True
    except Exception as e:
        pbar.close()
        print(f"Error downloading {filename}: {str(e)}")
        return False

def get_smos_data(start_date, end_date, lat_bounds, lon_bounds, username=None, password=None):
    """
    Retrieve SMOS soil moisture data using FTPS from ESA's dissemination service with progress tracking
    
    Parameters:
    -----------
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    lat_bounds : tuple
        (min_lat, max_lat) for region of interest
    lon_bounds : tuple
        (min_lon, max_lon) for region of interest
    username : str, optional
        ESA data portal username
    password : str, optional
        ESA data portal password
    
    Returns:
    --------
    xarray.Dataset
        Combined dataset with SMOS soil moisture data
    """
    print("\nInitializing SMOS FTPS connection...")
    
    # Get credentials if not provided
    if not username or not password:
        print("\nESA credentials required for SMOS data access.")
        print("Please use your EO-Sign In account credentials.")
        username = input("ESA username: ") if not username else username
        password = getpass.getpass("ESA password: ") if not password else password

    try:
        # Create SSL context
        ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLS_CLIENT)
        ssl_context.check_hostname = False
        ssl_context.verify_mode = ssl.CERT_NONE

        # Connect to SMOS FTPS server
        ftps = ftplib.FTP_TLS()
        ftps.ssl_version = ssl.PROTOCOL_TLS
        ftps.context = ssl_context
        
        # Connect with progress updates
        print("Connecting to SMOS FTPS server...")
        ftps.connect('smos-diss.eo.esa.int', 990)
        ftps.login(username, password)
        ftps.prot_p()
        
        print("Successfully connected to SMOS FTPS server")
        
        # Convert dates
        start_dt = pd.to_datetime(start_date)
        end_dt = pd.to_datetime(end_date)
        
        # Navigate to SMOS data directory
        base_path = '/SMOS/L2SM'
        ftps.cwd(base_path)
        
        # Collect all matching files first
        matching_files: List[Dict] = []
        
        # Calculate total directories to search
        years = range(start_dt.year, end_dt.year + 1)
        total_months = sum(12 if year != end_dt.year else end_dt.month 
                         for year in years)
        
        # Create progress bar for directory scanning
        with tqdm(total=total_months, desc="Scanning directories") as dir_pbar:
            for year in years:
                year_path = f"{year:04d}"
                try:
                    ftps.cwd(f"{base_path}/{year_path}")
                    
                    # Determine month range for current year
                    start_month = 1 if year != start_dt.year else start_dt.month
                    end_month = 12 if year != end_dt.year else end_dt.month
                    
                    for month in range(start_month, end_month + 1):
                        month_path = f"{month:02d}"
                        try:
                            ftps.cwd(f"{base_path}/{year_path}/{month_path}")
                            
                            # List files in directory
                            files = []
                            ftps.dir(files.append)
                            
                            # Filter files by date and type
                            for file_info in files:
                                if not file_info.startswith('d'):
                                    filename = file_info.split()[-1]
                                    if filename.endswith('.nc') or filename.endswith('.DBL'):
                                        try:
                                            file_date = pd.to_datetime(filename.split('_')[4][:8])
                                            if start_dt <= file_date <= end_dt:
                                                file_path = f"{base_path}/{year_path}/{month_path}/{filename}"
                                                matching_files.append({
                                                    'path': file_path,
                                                    'name': filename,
                                                    'date': file_date,
                                                    'size': get_file_size(ftps, filename)
                                                })
                                        except Exception:
                                            continue
                                            
                        except ftplib.error_perm:
                            print(f"Could not access month directory: {month_path}")
                        
                        dir_pbar.update(1)
                            
                except ftplib.error_perm:
                    print(f"Could not access year directory: {year_path}")
        
        if not matching_files:
            raise ValueError("No SMOS files found for the specified date range")
        
        print(f"\nFound {len(matching_files)} matching files")
        
        # Download and process files
        datasets = []
        total_size = sum(f['size'] for f in matching_files)
        
        with tempfile.TemporaryDirectory() as temp_dir:
            # Create main progress bar for overall download progress
            with tqdm(total=total_size, unit='B', unit_scale=True,
                     desc="Overall download progress") as main_pbar:
                
                for file_info in matching_files:
                    local_file = os.path.join(temp_dir, file_info['name'])
                    
                    # Download file with its own progress bar
                    if download_with_progress(ftps, file_info['path'], local_file):
                        try:
                            # Process file with progress update
                            with tqdm(total=1, desc=f"Processing {file_info['name']}", 
                                    leave=False) as proc_pbar:
                                ds = xr.open_dataset(local_file)
                                
                                # Select soil moisture variables
                                sm_vars = [var for var in ds.data_vars 
                                         if any(kw in var.lower() 
                                               for kw in ['sm', 'soil_moisture', 'moisture'])]
                                
                                if sm_vars:
                                    ds = ds[sm_vars]
                                    datasets.append(ds)
                                
                                proc_pbar.update(1)
                                
                        except Exception as e:
                            print(f"Failed to process {file_info['name']}: {str(e)}")
                            continue
                            
                    main_pbar.update(file_info['size'])
        
        ftps.quit()
        
        if not datasets:
            raise ValueError("No valid soil moisture data found in downloaded files")
        
        # Combine datasets with progress bar
        print("\nMerging datasets...")
        with tqdm(total=1, desc="Merging datasets") as merge_pbar:
            combined_ds = xr.merge(datasets)
            merge_pbar.update(1)
        
        # Print dataset information with formatted output
        print("\nDataset Information:")
        print("-" * 50)
        print(f"Dimensions: {dict(combined_ds.sizes)}")
        
        # Create progress bar for statistical analysis
        with tqdm(total=len(combined_ds.data_vars), 
                 desc="Calculating statistics") as stat_pbar:
            for var in combined_ds.data_vars:
                print(f"\nVariable: {var}")
                data = combined_ds[var].values
                valid_data = data[~np.isnan(data)]
                
                if len(valid_data) > 0:
                    print("Statistics:")
                    print(f"  Mean:     {np.nanmean(data):.4f}")
                    print(f"  Std Dev:  {np.nanstd(data):.4f}")
                    print(f"  Min:      {np.nanmin(data):.4f}")
                    print(f"  Max:      {np.nanmax(data):.4f}")
                    print(f"  Shape:    {data.shape}")
                    print(f"  Valid Points:    {len(valid_data):,}")
                    print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                    print(f"  Coverage:        {(len(valid_data) / data.size * 100):.1f}%")
                
                stat_pbar.update(1)
        
        return combined_ds
        
    except Exception as e:
        print(f"\nError: {str(e)}")
        raise

In [10]:
#sentinel-1 request
def get_sentinel1_data(start_date, end_date, lat_bounds, lon_bounds, max_files=10):
    """
    Retrieve Sentinel-1 soil moisture data with progress tracking
    
    Parameters:
    -----------
    start_date : str
        Start date in YYYY-MM-DD format
    end_date : str
        End date in YYYY-MM-DD format
    lat_bounds : tuple
        (min_lat, max_lat) for region of interest
    lon_bounds : tuple
        (min_lon, max_lon) for region of interest
    max_files : int
        Maximum number of files to retrieve
    
    Returns:
    --------
    xarray.Dataset
        Combined dataset with Sentinel-1 data
    """
    print("\nRetrieving Sentinel-1 data...")
    print(f"Time range: {start_date} to {end_date}")
    print(f"Spatial bounds: lat {lat_bounds}, lon {lon_bounds}")
    
    try:
        auth = earthaccess.login()
        
        granules = earthaccess.search_data(
            short_name="SENTINEL-1A_SLC",
            temporal=(start_date, end_date),
            bounding_box=(lon_bounds[0], lat_bounds[0], lon_bounds[1], lat_bounds[1]),
            count=max_files
        )
        
        if not granules:
            raise ValueError("No Sentinel-1 granules found")
        
        print(f"Found {len(granules)} granules")
        
        with tempfile.TemporaryDirectory() as temp_dir:
            downloaded_files = earthaccess.download(
                granules,
                local_path=temp_dir
            )
            
            if not downloaded_files:
                raise ValueError("Failed to download granules")
            
            datasets = []
            for file_path in tqdm(downloaded_files, desc="Processing files"):
                try:
                    ds = xr.open_dataset(file_path)
                    
                    # Select relevant variables (adjust based on actual variable names)
                    vars_of_interest = [var for var in ds.data_vars 
                                     if any(keyword in var.lower() 
                                           for keyword in ['vv', 'vh', 'angle', 'sigma'])]
                    if not vars_of_interest:
                        continue
                    ds = ds[vars_of_interest]
                    
                    # Apply spatial subsetting if coordinates exist
                    if 'latitude' in ds.dims:
                        ds = ds.sel(latitude=slice(lat_bounds[0], lat_bounds[1]))
                    if 'longitude' in ds.dims:
                        ds = ds.sel(longitude=slice(lon_bounds[0], lon_bounds[1]))
                    
                    datasets.append(ds)
                    
                except Exception as e:
                    print(f"Warning: Failed to process {os.path.basename(file_path)}: {str(e)}")
                    continue
            
            if not datasets:
                raise ValueError("No valid data found in downloaded files")
            
            combined_ds = xr.concat(datasets, dim='time')
            
            print("\nDataset Information:")
            print("-" * 50)
            print(f"Time range: {combined_ds.time.values[0]} to {combined_ds.time.values[-1]}")
            print(f"Number of timesteps: {len(combined_ds.time)}")
            print(f"Dimensions: {dict(combined_ds.sizes)}")
            
            for var in combined_ds.data_vars:
                print(f"\nVariable: {var}")
                data = combined_ds[var].values
                
                attrs = combined_ds[var].attrs
                units = attrs.get('units', 'unknown')
                long_name = attrs.get('long_name', var)
                
                print(f"Description: {long_name}")
                print(f"Units: {units}")
                print(f"Shape: {data.shape}")
                
                if np.issubdtype(data.dtype, np.number):
                    valid_data = data[~np.isnan(data)]
                    if len(valid_data) > 0:
                        print("Statistics:")
                        print(f"  Mean:     {np.nanmean(data):.4f}")
                        print(f"  Std Dev:  {np.nanstd(data):.4f}")
                        print(f"  Min:      {np.nanmin(data):.4f}")
                        print(f"  Max:      {np.nanmax(data):.4f}")
                        print(f"  Valid Points:    {len(valid_data):,}")
                        print(f"  Missing Points:  {np.sum(np.isnan(data)):,}")
                        print(f"  Coverage:        {(len(valid_data) / data.size * 100):.1f}%")
                    else:
                        print("No valid numeric data found")
            
            return combined_ds
            
    except Exception as e:
        print(f"\nError retrieving Sentinel-1 data: {str(e)}")
        raise

In [11]:
#Parameters
lat_bounds = (40.5, 45.02)
lon_bounds = (-79.77, -71.85)
start_date = "2023-01-01"
end_date = "2023-01-02"

In [ ]:
# ERA5_land request:
request = {
    "variable": "volumetric_soil_water_layer_1",
    "product_type": "reanalysis",
    "year": "2023",
    "month": "01",
    "day": ["01", "02"],
    "time": [f"{hour:02d}:00" for hour in range(24)],
    "area": [45.02, -79.77, 40.5, -71.85],  # [north, west, south, east]
    "format": "netcdf"
 }
    
    # Get the data
try:
     era5_data = get_era5_data(
        dataset="reanalysis-era5-land",
        request=request,
        output_file="era5_soil_moisture.nc"
    )  
except Exception as e:
    print(f"Failed to retrieve ERA5 data: {str(e)}")

In [ ]:
# Example usage NLDAS:
if __name__ == "__main__":
    start_date = "2023-01-01"
    end_date = "2023-01-02"
    lat_bounds = (40.5, 45.02)
    lon_bounds = (-79.77, -71.85)
    
    try:
        # Set up output file name
        output_file = f"nldas_data_{start_date}_{end_date}.nc"
        
        # Retrieve the data
        nldas_data = get_nldas_data(
            start_date=start_date,
            end_date=end_date,
            lat_bounds=lat_bounds,
            lon_bounds=lon_bounds
        )
        
        # Save the data
        print(f"\nSaving data to {output_file}...")
        nldas_data.to_netcdf(output_file)
        
        # Print file size
        file_size = os.path.getsize(output_file) / (1024 * 1024)  # Convert to MB
        print(f"File saved successfully. Size: {file_size:.2f} MB")
        
    except Exception as e:
        print(f"Failed to retrieve NLDAS data: {str(e)}")

In [ ]:
# GLDAS example
gldas_data = get_gldas_data(start_date, end_date, lat_bounds, lon_bounds)

In [ ]:
# FLDAS example
fldas_data = get_fldas_data(start_date, end_date, lat_bounds, lon_bounds)

In [ ]:
# SMAP example
smap_data = get_smap_data(start_date, end_date, lat_bounds, lon_bounds)

In [25]:
# MERRA-2 data 
merra2_data = get_merra2_data(
    bbox=(-79.77, 40.5, -71.85, 45.02),  # (min_lon, min_lat, max_lon, max_lat)
    date_range=('2020-01-01', '2020-01-31'),
    var_names=['SFMC', 'PRMC'] 
)

Job ID: 67239b288be99c3db6e2b0ad
Initial status: Accepted
Job status: Succeeded (100% complete)
Job Finished: Complete (M2T1NXLND_5.12.4)

Attempting download from: https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2T1NXLND.5.12.4/doc/MERRA2.README.pdf

Error downloading data: 404 Client Error: Not Found for url: https://goldsmr4.gesdisc.eosdis.nasa.gov/data/MERRA2/M2T1NXLND.5.12.4/doc/MERRA2.README.pdf%0D

Attempting download from: https://goldsmr4.gesdisc.eosdis.nasa.gov/opendap/MERRA2/M2T1NXLND.5.12.4/2020/01/MERRA2_400.tavg1_2d_lnd_Nx.20200101.nc4.nc4?SFMC[0:23][261:270][160:173],PRMC[0:23][261:270][160:173],time,lat[261:270],lon[160:173]

Successfully saved data to merra_2_soil_moisture_data.nc

=== Dataset Information ===
Dimensions: {'time': 24, 'lat': 10, 'lon': 14}

Variables: ['SFMC', 'PRMC']

Time range: 2020-01-01T00:30:00.000000000 to 2020-01-01T23:30:00.000000000
Spatial extent: -80.00°E to -71.88°E, 40.50°N to 45.00°N

=== Statistics for SFMC ===
Shape: (24, 10, 14)


/var/folders/w6/0tkgm2g11c9202ygt6n5r_qh0000gn/T/ipykernel_94552/3139272757.py:169: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"Dimensions: {dict(ds.dims)}")


In [30]:
smos_data=get_smos_data(start_date, end_date, lat_bounds, lon_bounds)


Initializing SMOS FTPS connection...

ESA credentials required for SMOS data access.
Please use your EO-Sign In account credentials.
Connecting to SMOS FTPS server...

Error: 


EOFError: 

In [ ]:
sentinel1_data=get_sentinel1_data(start_date, end_date, lat_bounds, lon_bounds, max_files=10)